In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Cell 1 – Install & Imports
!pip install --quiet torch torchvision webdataset tqdm pillow scikit-learn joblib matplotlib seaborn pyyaml

import os, sys, json, yaml, joblib
from pathlib import Path
import torch
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# set matplotlib style
plt.rcParams.update({"figure.max_open_warning": 0})


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.5 MB/s eta 0:00:00


In [3]:
# Cell 2 – Load Configuration & Paths (robust version)
import os
import yaml
from pathlib import Path

# Define default root paths
DEFAULT_ENV_PATHS = {
    "colab": "/content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project",
    "local": "/Users/stefanoroybisignano/Desktop/MLA/project/wsi-ssrl-rcc_project",
}

# Determine environment
IN_COLAB = Path("/content").exists()
PROJECT_ROOT = Path(
    os.getenv("PROJECT_ROOT", DEFAULT_ENV_PATHS["colab" if IN_COLAB else "local"])
).resolve()

# Path to YAML config (always in config/)
cfg_path = PROJECT_ROOT / "config" / "training.yaml"

# Check config file
if not cfg_path.exists():
    raise FileNotFoundError(f"❌ training.yaml not found at: {cfg_path}")

# Load YAML config
with cfg_path.open() as f:
    cfg = yaml.safe_load(f)

# Extract config values
EXP_CODE   = "20250723141202" #cfg.get("exp_code") or os.getenv("EXP_CODE") or "missing_code"
DATASET_ID = cfg["data"]["dataset_id"]

# Build central experiment path
EXP_DIR = PROJECT_ROOT / cfg["output"]["exp_dir"].format(
    dataset_id=DATASET_ID, exp_code=EXP_CODE
)

# Ovverride general training yaml file
cfg_path = EXP_DIR / f"training_{EXP_CODE}.yaml"
# Check config file
if not cfg_path.exists():
    raise FileNotFoundError(f"❌ training.yaml not found at: {cfg_path}")

# Load YAML config
with cfg_path.open() as f:
    cfg = yaml.safe_load(f)

# Display
print(f"📁 PROJECT_ROOT → {PROJECT_ROOT}")
print(f"📄 YAML loaded  → {cfg_path.name}")
print(f"🔑 EXP_CODE     → {EXP_CODE}")
print(f"📂 EXP_DIR      → {EXP_DIR}")


📁 PROJECT_ROOT → /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project
📄 YAML loaded  → training_20250723141202.yaml
🔑 EXP_CODE     → 20250723141202
📂 EXP_DIR      → /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/processed/dataset_9f30917e/experiments/20250723141202


In [4]:
# Cell 3 – Extend PYTHONPATH & Import Trainers + Utils

import sys
import importlib
from pathlib import Path

# Extend PYTHONPATH to include src/
SRC_DIR = PROJECT_ROOT / "src"
sys.path[:0] = [str(PROJECT_ROOT), str(SRC_DIR)]

# Import training utils from utils.training_utils
from utils.training_utils.registry import TRAINER_REGISTRY
from utils.training_utils.device_io import (
    choose_device,
    get_latest_checkpoint,
    load_checkpoint,
    save_json,
    save_joblib,
)
from utils.training_utils.data_utils import (
    build_loader,
    load_classifier,
    parse_label_from_filename,
)
from utils.training_utils.model_utils import mc_dropout_predictions
from utils.training_utils.metrics import (
    compute_classification_metrics,
    aggregate_fold_metrics,
    expected_calibration_error,
)

# Import trainer modules dynamically to ensure registration
trainer_names = ["simclr", "moco_v2", "rotation", "jepa", "supervised", "transfer"]
for name in trainer_names:
    try:
        importlib.import_module(f"trainers.{name}")
        print(f"✅ Imported trainer module: {name}")
    except ImportError as e:
        print(f"❌ Failed to import trainer {name}: {e}")

# Verify registration
for name in trainer_names:
    assert name in TRAINER_REGISTRY, f"❌ Missing trainer in registry: {name}"
print("📚 All trainers successfully registered.")

✅ Imported trainer module: simclr
✅ Imported trainer module: moco_v2
✅ Imported trainer module: rotation
✅ Imported trainer module: jepa
✅ Imported trainer module: supervised
✅ Imported trainer module: transfer
📚 All trainers successfully registered.


In [5]:
# Cell 4 – Evaluation Settings (debug-friendly)

device = choose_device()
eval_cfg = cfg.get("evaluation", {})

# 🔧 Debug: riduci MC_PASSES ed ECE_BINS per velocizzare
MC_PASSES = min(int(eval_cfg.get("mc_dropout_passes", 20)), 3)
ECE_BINS  = min(int(eval_cfg.get("ece_bins", 15)), 5)

GCAM_TOPK  = int(eval_cfg.get("gradcam", {}).get("top_k", 5))
GCAM_LAYER = eval_cfg.get("gradcam", {}).get("layer", None)

print(f"🖥️  Device:         {device}")
print(f"🔄  MC-dropout:     {MC_PASSES} passes")
print(f"📊  ECE bins:       {ECE_BINS}")
print(f"🔍  GradCAM++ top-k:{GCAM_TOPK}")
print(f"📐  GradCAM++ layer:{GCAM_LAYER}")


🖥️  Device:         cpu
🔄  MC-dropout:     3 passes
📊  ECE bins:       5
🔍  GradCAM++ top-k:5
📐  GradCAM++ layer:layer4


In [6]:
# Cell 5 – Helper Functions & Path Centralization

from pathlib import Path

def _paths(model_name: str, fold: int, patient_id: str = None) -> dict[str, Path]:
    """
    Build and return all relevant paths for a given model/fold,
    directly from the patterns in cfg['output'], WITHOUT formatting {epoch}.
    """
    # Base placeholders
    ph = {
        "dataset_id": DATASET_ID,
        "exp_code":   EXP_CODE,
        "model_name": model_name,
        "fold_idx":   fold,
        "patient_id": patient_id or "{patient_id}",
    }

    # Experiment directories
    ph["exp_dir"]       = cfg["output"]["exp_dir"].format(**ph)
    ph["exp_model_dir"] = cfg["output"]["exp_model_dir"].format(**ph)

    out: dict[str, Path] = {}

    # ─── Training ────────────────────────────────────────────────────────
    out["ckpt_dir"] = PROJECT_ROOT / ph["exp_model_dir"] / f"fold{fold}" / "training"

    t = cfg["output"]["training"]
    out["features_train"] = PROJECT_ROOT / t["features"].format(**ph)
    out["clf"]            = PROJECT_ROOT / t["clf"].format(**ph)
    out["scaler"]         = PROJECT_ROOT / t["scaler"].format(**ph)
    out["loss_json"]      = PROJECT_ROOT / t["loss_json"].format(**ph)
    out["log"]            = PROJECT_ROOT / t["log"].format(**ph)

    # ─── Inference ───────────────────────────────────────────────────────
    i = cfg["output"]["inference"]
    out["patch_preds"]   = PROJECT_ROOT / i["patch_preds"].format(**ph)
    out["patient_preds"] = PROJECT_ROOT / i["patient_preds"].format(**ph)
    out["mc_logits"]     = PROJECT_ROOT / i["mc_logits"].format(**ph)
    out["metrics"]       = PROJECT_ROOT / i["metrics"].format(**ph)

    # ─── Explainability ──────────────────────────────────────────────────
    e = cfg["output"]["explain"]
    out["gradcam_dir"]  = PROJECT_ROOT / e["gradcam_dir"].format(**ph)
    out["metadata_csv"] = PROJECT_ROOT / e["metadata_csv"].format(**ph)

    # ─── Aggregation ─────────────────────────────────────────────────────
    a = cfg["output"]["aggregate"]
    out["agg_metrics"] = PROJECT_ROOT / a["metrics"].format(**ph)
    out["agg_summary"] = PROJECT_ROOT / a["summary_img"].format(**ph)

    # ─── Experiment-Level ────────────────────────────────────────────────
    x = cfg["output"]["experiment_level"]
    out["exp_json"] = PROJECT_ROOT / x["comparison_json"].format(**ph)
    out["exp_img"]  = PROJECT_ROOT / x["comparison_img"].format(**ph)

    # ─── Ensure directories exist ────────────────────────────────────────
    for key, p in out.items():
        if "dir" in key:
            p.mkdir(parents=True, exist_ok=True)
        else:
            p.parent.mkdir(parents=True, exist_ok=True)

    return out

def _completed(paths: dict[str, Path], is_ssl: bool) -> bool:
    """
    Returns True if all the necessary inference artifacts for this fold
    are already on disk, so we can skip evaluation.
    """
    # Always require patch‐level preds + metrics JSON
    required = ["patch_preds", "metrics"]
    # For SSL models also require MC logits and patient‐level CSV
    if is_ssl:
        required += ["mc_logits", "patient_preds"]
    return all(paths[k].exists() for k in required)


def extract_patient_id(key: str) -> str:
    """
    Extract patient ID from a key formatted like 'CLASS_HPxxxx_x_y'.
    """
    parts = key.split("_")
    return next((p for p in parts if p.startswith(("HP", "H"))), "UNKNOWN")


def ece(probs, labels):
    """
    Expected Calibration Error (ECE) helper for quick access.
    """
    return expected_calibration_error(probs, labels, n_bins=ECE_BINS)

In [7]:
# Cell 6a – Core Evaluation Functions

import torch
import torch.nn as nn
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from collections import defaultdict, Counter

from sklearn.metrics import (
    accuracy_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report
)

from utils.training_utils.metrics import (
    TemperatureScaler,
    expected_calibration_error,
    mc_dropout_statistics
)
from utils.training_utils.data_utils import load_classifier, parse_label_from_filename
from utils.training_utils.device_io import (
    get_latest_checkpoint,
    load_checkpoint,
    save_json
)
from utils.training_utils.registry import TRAINER_REGISTRY
from utils.training_utils.model_utils import mc_dropout_predictions

def _fix_fc_from_ckpt(model: torch.nn.Module, ckpt_path: Path) -> None:
    """
    Replace model.fc so that out_features matches the checkpoint metadata.

    Parameters
    ----------
    model : torch.nn.Module
        The backbone with a (possibly wrong) final fully-connected layer.
    ckpt_path : Path
        Path to the .pt checkpoint; must contain 'class_to_idx' metadata.
    """
    ckpt_data = torch.load(ckpt_path, map_location="cpu")
    class_to_idx = ckpt_data.get("class_to_idx", {})
    n_classes = len(class_to_idx)
    if n_classes <= 0:
        raise ValueError("Checkpoint lacks valid 'class_to_idx' metadata")
    if not hasattr(model, "fc"):
        raise AttributeError("Model has no .fc attribute to patch")
    if model.fc.out_features != n_classes:
        in_f = model.fc.in_features
        model.fc = nn.Linear(in_f, n_classes)
        print(f"🔧 Patched fc layer: in_features={in_f} → out_features={n_classes}")

def load_model_and_components(model_name: str, fold: int):
    """
    Carica:
      - il modello PyTorch dal checkpoint
      - per SSL: il probe (clf + label encoder) e, se presente, il TemperatureScaler

    Restituisce:
      model        : nn.Module      – backbone/encoder pronto per inferenza
      is_ssl       : bool           – True se modello SSL
      clf          : Any            – classificatore (probe) o None per SL
      le           : Any            – LabelEncoder per SSL, altrimenti None
      temp_scaler  : TemperatureScaler | None
    """
    paths   = _paths(model_name, fold)
    cfg_m   = cfg["models"][model_name]
    is_ssl  = cfg_m["type"] == "ssl"
    trainer = TRAINER_REGISTRY[model_name](cfg_m, cfg["data"])

    # 1️⃣ Scegli checkpoint
    if is_ssl and cfg.get("train_encoder_once", False) and fold > 0:
        ckpt = get_latest_checkpoint(_paths(model_name, 0)["ckpt_dir"])
        if ckpt is None:
            raise FileNotFoundError(f"❌ Nessun checkpoint trovato in fold0 per {model_name}")
        print(f"   ➔ SSL+train_encoder_once → encoder da fold0: {ckpt.name}")
    else:
        ckpt = get_latest_checkpoint(paths["ckpt_dir"])
        if ckpt is None:
            raise FileNotFoundError(f"❌ Nessun checkpoint trovato in fold{fold} per {model_name}")
        print(f"   ➔ Caricamento checkpoint fold{fold} → {ckpt.name}")

    # 2️⃣ Carica pesi nel modello
    if is_ssl:
        full_model, _ = trainer.get_resume_model_and_optimizer()
        load_checkpoint(ckpt, model=full_model)
        feat_mod = getattr(trainer, "encoder", None) or getattr(trainer, "model", None)
        if feat_mod is None:
            raise AttributeError(f"No feature submodule found on {trainer}")
        model = feat_mod.to(device).eval()
    else:
        # Patch dinamica del layer fc per SL/transfer
        _fix_fc_from_ckpt(trainer.model, ckpt)
        load_checkpoint(ckpt, model=trainer.model)
        model = trainer.model.to(device).eval()

    # 3️⃣ Carica probe / classifier / calibratore
    clf = le = temp_scaler = None
    if is_ssl:
        # classificatore joblib salvato in training SSL
        clf, le = load_classifier(paths["clf"])
        # eventualmente temperature scaling
        scaler_path = paths["scaler"]
        if scaler_path.exists():
            obj = joblib.load(scaler_path)
            if isinstance(obj, TemperatureScaler):
                temp_scaler = obj
                print("   ➔ Loaded TemperatureScaler (calibrate probs)")
        else:
            print(f"   ➔ No TemperatureScaler for {model_name} fold {fold}")

    # 4️⃣ Debug printout
    print(f"   ➔ SSL pipeline?  {is_ssl}")
    print(f"   ➔ Classifier?    {clf is not None}")
    print(f"   ➔ Temp-scaler?   {temp_scaler is not None}")

    return model, is_ssl, clf, le, temp_scaler


In [8]:
#  Cell 6b – Core Evaluation Functions
def run_patch_inference(model, loader, is_ssl, clf, temp_scaler):
    """
    Inferenzia patch-level. Restituisce:
      keys (dummy), y_true, y_pred, probs (calibrated if temp_scaler)
    """
    keys, y_true, y_pred = [], [], []
    probs_list = []

    for batch_idx, batch in enumerate(tqdm(loader, desc="Patches")):
        imgs, labels = batch
        imgs = imgs.to(device)

        if is_ssl:
            with torch.no_grad():
                feats = model(imgs).cpu().numpy()
            raw_p = clf.predict_proba(feats)
        else:
            with torch.no_grad():
                logits = model(imgs)
                raw_p = torch.softmax(logits, dim=1).cpu().numpy()

        # calibration
        if temp_scaler is not None:
            logits_for_cal = np.log(raw_p + 1e-12)
            p = temp_scaler.transform_proba(logits_for_cal)
        else:
            p = raw_p

        preds = p.argmax(axis=1)
        t     = labels.cpu().numpy()

        y_true.extend(t.tolist())
        y_pred.extend(preds.tolist())
        probs_list.append(p)

        print(f"   • Batch {batch_idx} done.")

    probs = np.vstack(probs_list)
    return keys, np.array(y_true), np.array(y_pred), probs


def save_patch_outputs(model_name, fold, keys, y_true, y_pred, probs):
    torch.save({
        "keys": keys,
        "true": y_true,
        "pred": y_pred,
        "probs": probs
    }, _paths(model_name, fold)["patch_preds"])


def save_mc_logits(model_name, fold, model, loader):
    mc = mc_dropout_predictions(model, loader, device=device, T=MC_PASSES)
    np.save(_paths(model_name, fold)["mc_logits"], mc)
    return mc


def compute_and_save_metrics(model_name, fold, y_true, y_pred, probs, mc=None):
    """
    Calcola tutte le metriche, MC-stats e ECE post-calibrazione.
    """
    mc_stats = mc_dropout_statistics(mc) if mc is not None else {}

    acc  = accuracy_score(y_true, y_pred)
    f1   = f1_score(y_true, y_pred, average="macro")
    try:
        auc = roc_auc_score(y_true, y_pred, average="macro", multi_class="ovo")
    except ValueError:
        auc = None
    cm = confusion_matrix(y_true, y_pred).tolist()
    cr = classification_report(y_true, y_pred, output_dict=True)

    mets = {
        "accuracy": acc,
        "macro_f1": f1,
        "roc_auc": auc,
        "confusion_matrix": cm,
        "class_report": cr,
        **mc_stats
    }

    # ECE post-calibrazione
    mets["ece_post"] = expected_calibration_error(probs, y_true, n_bins=ECE_BINS)

    save_json(mets, _paths(model_name, fold)["metrics"])
    return mets


In [9]:
# Cell 7 – Per-Fold Evaluation (aggiornato)

from utils.training_utils.data_utils import default_transforms, build_loader, KNOWN_LABELS, parse_label_from_filename
from utils.training_utils.device_io import get_latest_checkpoint
from collections import defaultdict, Counter
import numpy as np
import pandas as pd

def aggregate_patient_results(model_name, fold, keys, y_pred, probs):
    """
    Aggrega predizioni per paziente (majority voting) e salva CSV.
    """
    by_pt = defaultdict(list)
    for k, y, conf in zip(keys, y_pred, probs.max(axis=1)):
        pid = extract_patient_id(k)
        by_pt[pid].append((y, conf))

    rows = []
    for pid, recs in by_pt.items():
        votes, confs = zip(*recs)
        rows.append({
            "patient_id": pid,
            "true_label": parse_label_from_filename(pid),
            "pred_label": Counter(votes).most_common(1)[0][0],
            "n_patches": len(recs),
            "mean_conf_raw": float(np.mean(confs))
        })

    pd.DataFrame(rows).to_csv(
        _paths(model_name, fold)["patient_preds"], index=False
    )

def evaluate_fold(model_name: str, fold: int):
    print(f"\n🔍 Evaluating {model_name} fold {fold}…")

    # 0️⃣ Paths e checkpoint
    paths = _paths(model_name, fold)
    ckpt  = get_latest_checkpoint(paths["ckpt_dir"])
    if ckpt is None:
        print(f"⚠️ Nessun checkpoint trovato per {model_name} fold {fold}, salto valutazione")
        return

    # 1️⃣ Skip se artefatti già presenti
    is_ssl = (cfg["models"][model_name]["type"] == "ssl")
    if _completed(paths, is_ssl):
        print(f"⚡ Skipping {model_name} fold {fold}: artifacts already present")
        return

    # 2️⃣ Carica modello, probe, scaler
    print("📥 Loading model, classifier, scaler…")
    model, is_ssl, clf, le, temp_scaler = load_model_and_components(model_name, fold)

    # 3️⃣ Build test loader
    patch_size = cfg["models"][model_name].get("patch_size", 224)
    batch_size = cfg["models"][model_name]["training"]["batch_size"]
    test_rel   = cfg["data"]["test"].format(
        fold_idx=fold,
        dataset_id=cfg["data"]["dataset_id"]
    )
    test_wds   = (PROJECT_ROOT / test_rel).resolve()
    print(f"🧪 Using test shard: {test_wds}")
    assert test_wds.exists(), f"❌ Test shard not found: {test_wds}"

    # ❗ mappatura globale delle classi
    all_classes  = sorted(KNOWN_LABELS)  # {"ccRCC","pRCC","CHROMO","ONCO","not_tumor"} :contentReference[oaicite:0]{index=0}
    class_to_idx = {cls: i for i, cls in enumerate(all_classes)}

    loader = build_loader(
        str(test_wds),
        class_to_idx=class_to_idx,
        patch_size=patch_size,
        batch_size=batch_size,
        device=device,
        augment=False,
    )
    print(f"📦 DataLoader ready with batch_size = {batch_size}")

    # 4️⃣ Patch-level inference
    print("▶️ Running inference (patch-level)…")
    keys, y_true, y_pred, probs = run_patch_inference(
        model, loader, is_ssl, clf, temp_scaler
    )
    print(f"   ➔ Patches processed: {len(y_true)}")
    save_patch_outputs(model_name, fold, keys, y_true, y_pred, probs)
    print("💾 Saved patch-level outputs.")

    # 5️⃣ MC-Dropout (solo SSL)
    if is_ssl:
        print("🔄 Running MC-Dropout…")
        mc = save_mc_logits(model_name, fold, model, loader)
        print("💾 Saved MC-Dropout logits.")
    else:
        mc = None

    # 6️⃣ Compute & save metrics
    print("📊 Computing metrics…")
    compute_and_save_metrics(model_name, fold, y_true, y_pred, probs, mc)
    print("💾 Saved metrics JSON.")

    # 7️⃣ Patient-level aggregation (solo SSL)
    if is_ssl:
        print("👨‍⚕️ Aggregating patient-level results…")
        aggregate_patient_results(model_name, fold, keys, y_pred, probs)
        print("💾 Saved patient-level CSV.")

    print(f"✅ Done {model_name} fold {fold}\n")


# 🚀 Run evaluation for all models and folds
for model_name in cfg["run_models"]:
    for fold_idx in cfg["folds"]:
        evaluate_fold(model_name, fold_idx)



🔍 Evaluating rotation fold 0…
📥 Loading model, classifier, scaler…
   ➔ Caricamento checkpoint fold0 → RotationTrainer_bestepoch001.pt
   ➔ Loaded TemperatureScaler (calibrate probs)
   ➔ SSL pipeline?  True
   ➔ Classifier?    True
   ➔ Temp-scaler?   True
🧪 Using test shard: /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/processed/dataset_9f30917e/webdataset/test_holdout.tar
📦 DataLoader ready with batch_size = 64
▶️ Running inference (patch-level)…


Patches: 1it [00:34, 34.10s/it]

   • Batch 0 done.


Patches: 2it [01:08, 34.13s/it]

   • Batch 1 done.


...

   • Batch 7 done.
   ➔ Patches processed: 487
💾 Saved patch-level outputs.
🔄 Running MC-Dropout…
🔁 MC-Dropout pass 1/3


    • Batch 1 / MC-pass 1
    • Batch 2 / MC-pass 1
    • Batch 3 / MC-pass 1
    • Batch 4 / MC-pass 1
    • Batch 5 / MC-pass 1
    • Batch 6 / MC-pass 1
    • Batch 7 / MC-pass 1
    • Batch 8 / MC-pass 1
🔁 MC-Dropout pass 2/3
    • Batch 1 / MC-pass 2
    • Batch 2 / MC-pass 2
    • Batch 3 / MC-pass 2
    • Batch 4 / MC-pass 2
    • Batch 5 / MC-pass 2
    • Batch 6 / MC-pass 2
    • Batch 7 / MC-pass 2
    • Batch 8 / MC-pass 2
🔁 MC-Dropout pass 3/3
    • Batch 1 / MC-pass 3
    • Batch 2 / MC-pass 3
    • Batch 3 / MC-pass 3
    • Batch 4 / MC-pass 3
    • Batch 5 / MC-pass 3
    • Batch 6 / MC-pass 3
    • Batch 7 / MC-pass 3
    • Batch 8 / MC-pass 3
✅ MC-Dropout completato: 3 pass, 487 patch per pass.
💾 Saved MC-Dropout logits.
📊 Computing metrics…
💾 Saved metrics JSON.
👨‍⚕️ Aggregating patient-level results…
💾 Saved patient-level CSV.
✅ Done simclr fold 0


🔍 Evaluating simclr fold 1…
⚠️ Nessun checkpoint trovato per simclr fold 1, salto valutazione

🔍 Evaluating moco_v2 fol

/usr/local/lib/python3.11/dist-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


   ➔ Caricamento checkpoint fold0 → MoCoV2Trainer_bestepoch001.pt


AttributeError: No feature submodule found on <trainers.moco_v2.MoCoV2Trainer object at 0x7c8f8cbe9810>

In [ ]:
!ls /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/processed/dataset_9f30917e/webdataset/


In [ ]:
# Cell 8 – Fold-Level & Experiment-Level Aggregation (robust)

import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from utils.training_utils.device_io import save_json
from utils.training_utils.metrics import aggregate_fold_metrics

def aggregate_model(model_name: str):
    """
    Raccoglie e aggrega le metriche per ciascun fold disponibile.
    Salta automaticamente i fold senza file di metrics.
    """
    # 1️⃣ Raccogli le metriche per fold esistenti
    all_m = []
    available_folds = []
    for f in cfg["folds"]:
        metrics_path = Path(_paths(model_name, f)["metrics"])
        if metrics_path.exists():
            all_m.append(json.load(open(metrics_path)))
            available_folds.append(f)
        else:
            print(f"⚠️ Nessuna metrica trovata per {model_name} fold {f}, skip")

    if not all_m:
        print(f"❌ Nessuna metrica disponibile per modello '{model_name}', skip aggregation")
        return

    # 2️⃣ Estrai solo i campi numerici (int/float) dal primo fold valido
    numeric_keys = [k for k, v in all_m[0].items() if isinstance(v, (int, float))]

    # 3️⃣ Costruisci lista di dict solo con quei campi, trasformando None→NaN
    numeric_per_fold = []
    for m in all_m:
        filtered = {k: (m.get(k) if isinstance(m.get(k), (int, float)) else np.nan)
                    for k in numeric_keys}
        numeric_per_fold.append(filtered)

    # 4️⃣ Debug: segnala NaN trovati
    print(f"\n🔍 Debug NaN per modello '{model_name}':")
    nan_found = False
    for idx, m in enumerate(numeric_per_fold):
        f = available_folds[idx]
        for k in numeric_keys:
            if pd.isna(m[k]):
                print(f"  ⚠️ NaN in fold {f}, metrica '{k}'")
                nan_found = True
    if not nan_found:
        print("  ✅ Nessun NaN rilevato nelle metriche numeriche.")

    # 5️⃣ Aggrega mean±std e salva JSON
    summary = aggregate_fold_metrics(numeric_per_fold)
    agg_metrics_path = Path(_paths(model_name, available_folds[0])["agg_metrics"])
    save_json(summary, agg_metrics_path)
    print(f"💾 Saved aggregated metrics → {agg_metrics_path}")

    # 6️⃣ Heatmap delle medie
    dfm = pd.DataFrame(numeric_per_fold, index=available_folds)
    fig, ax = plt.subplots(figsize=(6, 4))
    sns.heatmap(
        dfm.mean()[numeric_keys].to_frame().T,
        annot=True, fmt=".3f", ax=ax
    )
    fig.savefig(Path(_paths(model_name, available_folds[0])["agg_summary"]), bbox_inches="tight")
    plt.close(fig)
    print(f"✅ Aggregated {model_name}")

def aggregate_experiment():
    """
    Costruisce il riepilogo tra modelli, include solo i modelli con JSON agg esistenti.
    """
    rows = []
    for m in cfg["run_models"]:
        exp_json_path = Path(_paths(m, cfg["folds"][0])["agg_metrics"])
        if exp_json_path.exists():
            met = json.load(open(exp_json_path))
            row = {"model": m}
            for k, v in met.items():
                row[k] = v["mean"]
            rows.append(row)
        else:
            print(f"⚠️ Saltato modello '{m}' in experiment-level perché manca agg_metrics")

    if not rows:
        print("❌ Nessun modello ha agg_metrics, skip experiment aggregation")
        return

    # 1️⃣ Salva JSON di esperimento
    exp_json_path = Path(_paths(cfg["run_models"][0], cfg["folds"][0])["exp_json"])
    save_json(rows, exp_json_path)
    print(f"💾 Saved experiment summary JSON → {exp_json_path}")

    # 2️⃣ Barplot accuracy
    df = pd.DataFrame(rows)
    fig, ax = plt.subplots(figsize=(8, 4))
    sns.barplot(data=df, x="model", y="accuracy", ax=ax)
    ax.set_title("Model Accuracy Comparison")
    fig.savefig(Path(_paths(cfg["run_models"][0], cfg["folds"][0])["exp_img"]), bbox_inches="tight")
    plt.close(fig)
    print("✅ Experiment-level comparison complete")

# 🚀 Esegui aggregazioni
for m in cfg["run_models"]:
    aggregate_model(m)
aggregate_experiment()
